In [ ]:
# Student ID: F319859
"""
Module: menu.ipynb
Description: This Jupyter Notebook serves as the main graphical user interface for the video game rental management system. It integrates functionalities from various modules (gameSearch, gameRent, gameReturn, inventoryPruning) and provides a user-friendly interface for store managers to manage game rentals, returns, and inventory. The interface includes options to search for games, rent out games, return games, collect feedback, and prune the inventory based on rental frequency or ratings.
"""
import sys
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Add the modules directory to the system path
sys.path.append('modules/')
from modules import gameSearch 
from modules import gameRent
from modules import gameReturn
from modules import inventoryPruning

# Styling for buttons
button_style = {'description_width': 'initial', 'button_color': 'lightblue'}

# Define widgets for the interactive interface
searchCriteriaDropdown = widgets.Dropdown(options=['Title', 'Genre', 'Platform'], description='Search by:')
searchQueryInput = widgets.Text(description="Search For:")
searchButton = widgets.Button(description="Search Games", style=button_style)

customerIdInputForRent = widgets.Text(description="Customer ID:")
gameIdInputForRent = widgets.Text(description="Game ID:")
rentButton = widgets.Button(description="Rent Game", style=button_style)

gameIdInputForReturn = widgets.Text(description="Game ID:")
ratingInput = widgets.IntSlider(value=3, min=1, max=5, step=1, description='Rating:')
commentsInput = widgets.Textarea(description="Comments:")
returnButton = widgets.Button(description="Return Game", style=button_style)

pruneButton = widgets.Button(description="Prune Games", style=button_style)
gamesToRemoveInput = widgets.Textarea(description="Enter Game IDs to Remove:")
confirmPruneButton = widgets.Button(description="Confirm Removal", style=button_style)

output = widgets.Output()

# Define layout for widgets
layout = widgets.Layout(width='auto', margin='10px')
for widget in [searchCriteriaDropdown, searchQueryInput, customerIdInputForRent, gameIdInputForRent,
               gameIdInputForReturn, ratingInput, commentsInput, gamesToRemoveInput]:
    widget.layout = layout

# Function to format and display search results in HTML format
def format_search_results_to_html(results):
    # Format search results as an HTML table with a simpler design
    if not results:
        return "No results found."

    # Styles for the table, headers, and rows
    table_style = "border-collapse: collapse; width: 100%;"
    header_style = "background-color: #4CAF50; color: white; text-align: left; padding: 8px;"
    cell_style = "border: 1px solid #ddd; padding: 8px;"

    html_output = f"<table style='{table_style}'>"
    html_output += f"<tr style='{header_style}'>"
    html_output += "<th>ID</th><th>Platform</th><th>Genre</th><th>Title</th><th>Publisher</th><th>Purchase Date</th></tr>"

    for game in results:
        row_style = cell_style if results.index(game) % 2 == 0 else f"{cell_style} background-color: #f2f2f2;"
        html_output += f"<tr style='{row_style}'>"
        for data in game:
            html_output += f"<td style='{cell_style}'>{data}</td>"
        html_output += "</tr>"

    html_output += "</table>"
    return html_output


# Define callback functions for buttons
# Callback for Search button
def on_search_button_clicked(b):
    with output:
        clear_output()
        searchQuery = searchQueryInput.value
        if searchQuery:
            searchFunction = {
                'Title': gameSearch.searchByTitle,
                'Genre': gameSearch.searchByGenre,
                'Platform': gameSearch.searchByPlatform
            }.get(searchCriteriaDropdown.value, lambda query: [])
            searchResults = searchFunction(searchQuery)
            display(HTML(format_search_results_to_html(searchResults)))
        else:
            print("Please enter a search query.")


# Callback for Rent button
def on_rent_button_clicked(b):
    with output:
        clear_output()
        customerId = customerIdInputForRent.value
        gameId = gameIdInputForRent.value
        if customerId and gameId:
            rentResult = gameRent.rentGame(customerId, gameId)
            print(rentResult)
        else:
            print("Please enter both Customer ID and Game ID.")

# Callback for Return button
def on_return_button_clicked(b):
    with output:
        clear_output()
        gameId = gameIdInputForReturn.value
        rating = ratingInput.value
        comments = commentsInput.value
        if gameId:
            returnResult = gameReturn.returnGame(gameId, rating, comments)
            print(returnResult)
        else:
            print("Please enter a Game ID.")

# Callback for Prune button
def on_prune_button_clicked(b):
    with output:
        clear_output()
        suggestedGamesForRemoval = inventoryPruning.pruneGames()
        if suggestedGamesForRemoval:
            print("Games suggested for removal due to low renting:")
            for game_id in suggestedGamesForRemoval:
                print(game_id)
            display(gamesToRemoveInput, confirmPruneButton)
        else:
            print("No games are suggested for removal based on the frequency criteria.")

# Callback for Confirm Removal button
def on_confirm_prune_button_clicked(b):
    with output:
        clear_output()
        gamesToRemove = [game.strip() for game in gamesToRemoveInput.value.split(',') if game.strip()]
        removalResult = inventoryPruning.removeSelectedGames(gamesToRemove)
        print(removalResult)

# Linking buttons to callback functions
searchButton.on_click(on_search_button_clicked)
rentButton.on_click(on_rent_button_clicked)
returnButton.on_click(on_return_button_clicked)
pruneButton.on_click(on_prune_button_clicked)
confirmPruneButton.on_click(on_confirm_prune_button_clicked)

# Organize widgets into sections using Accordion
accordion = widgets.Accordion(children=[widgets.VBox([searchCriteriaDropdown, searchQueryInput, searchButton]),
                                        widgets.VBox([customerIdInputForRent, gameIdInputForRent, rentButton]),
                                        widgets.VBox([gameIdInputForReturn, ratingInput, commentsInput, returnButton]),
                                        widgets.VBox([pruneButton, gamesToRemoveInput, confirmPruneButton])])
accordion.set_title(0, 'Search Games')
accordion.set_title(1, 'Rent Game')
accordion.set_title(2, 'Return Game')
accordion.set_title(3, 'Prune Inventory')

# Display the organized layout along with the output widget
display(accordion, output)